In [ ]:
import numpy as np
from scipy.stats import skew, kurtosis
from scipy.optimize import minimize

In [ ]:
# Define asset returns
returns = np.random.normal(0.05, 0.2, (10, 252*10))  # 10 assets, 10 years of daily returns

# Calculate expected returns and covariance matrix
exp_returns = np.mean(returns, axis=1)
cov_matrix = np.cov(returns)

# Calculate skewness and kurtosis for each asset
skews = skew(returns, axis=1)
kurts = kurtosis(returns, axis=1)

# Augment the covariance matrix based on skewness and kurtosis
skewness_matrix = np.diag(skews)  # diagonal matrix of skewness values
kurtosis_matrix = np.diag(kurts)  # diagonal matrix of kurtosis values
cov_matrix_augmented = cov_matrix + 0.5 * skewness_matrix + 0.1 * kurtosis_matrix

# Calculate optimal portfolio using MPT with augmented covariance matrix

def calculate_portfolio_variance(weights, exp_returns, cov_matrix):
    port_return = np.sum(exp_returns * weights)
    port_var = np.dot(weights.T, np.dot(cov_matrix, weights))
    return port_var

def calculate_portfolio_return(weights, exp_returns):
    port_return = np.sum(exp_returns * weights)
    return port_return

def check_sum(weights):
    return np.sum(weights) - 1

def optimize_portfolio(exp_returns, cov_matrix):
    num_assets = len(exp_returns)
    args = (exp_returns, cov_matrix)

    constraints = ({'type': 'eq', 'fun': check_sum})
    bounds = tuple((0, 1) for asset in range(num_assets))

    initial_guess = num_assets * [1. / num_assets]
    optimal_portfolio = minimize(calculate_portfolio_variance, initial_guess, args=args, method='SLSQP',
                                  bounds=bounds, constraints=constraints)
    return optimal_portfolio.x

optimal_weights = optimize_portfolio(exp_returns, cov_matrix_augmented)
optimal_return = calculate_portfolio_return(optimal_weights, exp_returns)
optimal_variance = calculate_portfolio_variance(optimal_weights, exp_returns, cov_matrix_augmented)
